In [10]:
from pathlib import Path
from argparse import ArgumentParser
import numpy as np
from rasterio.warp import reproject, Resampling
from rasterio.features import rasterize

import geopandas as gpd
import rasterio

from gridfinder import clip_raster
import json
from rasterio.mask import mask
import gridfinder as gf


def make_same_as(curr_arr, curr_aff, curr_crs, dest_arr_like, dest_affine, dest_crs):
    dest_arr = np.empty_like(dest_arr_like)

    with rasterio.Env():
        reproject(
            source=curr_arr,
            destination=dest_arr,
            src_transform=curr_aff,
            dst_transform=dest_affine,
            src_crs=curr_crs,
            dst_crs=dest_crs,
            resampling=Resampling.nearest,
        )

    return dest_arr

def make_same_as_dem(curr_arr, curr_aff, curr_crs, dest_arr_like, dest_affine, dest_crs):
    dest_arr = np.empty_like(dest_arr_like)

    with rasterio.Env():
        reproject(
            source=curr_arr,
            destination=dest_arr,
            src_transform=curr_aff,
            dst_transform=dest_affine,
            src_crs=curr_crs,
            dst_crs=dest_crs,
            resampling=Resampling.max-Resampling.min,
        )

    return dest_arr


def clip_all(raster_in, admin_in, raster_shape_dir, dir_out, code="ADM0_A3"):
    #Raster in is the one thing we want to clip, needs to be a raster
    #admin in is the boundaries
    #Raster_shape dir is the targets dir for the correct shapes
    #dir_out is what we want to call this dir 
    raster_in = Path(raster_in).expanduser()
    admin_in = Path(admin_in).expanduser()
    raster_shape_dir = Path(raster_shape_dir).expanduser()
    dir_out = Path(dir_out).expanduser()

    admin = gpd.read_file(admin_in)
    countries = admin[code].tolist()

    for c in countries:
        print(f"Doing {c}")
        c_out = dir_out / f"{c}.tif"

        aoi = admin[admin[code] == c]
        try:
            arr, aff, crs = gf.clip_raster(raster_in, aoi)
        except ValueError:
            print("No input data for this AOI - skipped")
            continue
        targets_in = raster_shape_dir / f"{c}.tif"
        targets_rd = rasterio.open(targets_in)
        dest_arr_like = targets_rd.read(1)
        dest_affine = targets_rd.transform
        dest_crs = targets_rd.crs

        new_arr = make_same_as(arr, aff, crs, dest_arr_like, dest_affine, dest_crs)

        gf.save_raster(c_out, new_arr, dest_affine, dest_crs)
        
        
#Raster is just the shapes

In [11]:

def prepare_roads(roads_in, aoi_in, ntl_in, motorway, trunk, primary, secondary, tertiary, unclassified, residential, service):
    ntl_rd = rasterio.open(ntl_in)
    shape = ntl_rd.read(1).shape
    affine = ntl_rd.transform

    if isinstance(aoi_in, gpd.GeoDataFrame):
        aoi = aoi_in
    else:
        aoi = gpd.read_file(aoi_in)
    roads_masked = gpd.read_file(roads_in, mask=aoi)
    roads = gpd.sjoin(roads_masked, aoi, how="inner", op="intersects")
    roads = roads[roads_masked.columns]

    roads["weight"] = 1
    roads.loc[roads["highway"] == "motorway", "weight"] = motorway
    roads.loc[roads["highway"] == "trunk", "weight"] = trunk
    roads.loc[roads["highway"] == "primary", "weight"] = primary
    roads.loc[roads["highway"] == "secondary", "weight"] = secondary
    roads.loc[roads["highway"] == "tertiary", "weight"] = tertiary
    roads.loc[roads["highway"] == "unclassified", "weight"] = unclassified
    roads.loc[roads["highway"] == "residential", "weight"] = residential
    roads.loc[roads["highway"] == "service", "weight"] = service

    # Power lines get weight 0
    if "power" in roads:
        roads.loc[roads["power"] == "line", "weight"] = 0

    roads = roads[roads.weight != 1]

    # sort by weight descending so that lower weight (bigger roads) are
    # processed last and overwrite higher weight roads
    roads = roads.sort_values(by="weight", ascending=False)

    roads_for_raster = [(row.geometry, row.weight) for _, row in roads.iterrows()]
    roads_raster = rasterize(
        roads_for_raster,
        out_shape=shape,
        fill=1,
        default_value=0,
        all_touched=True,
        transform=affine,
    )

    return roads_raster, affine





In [16]:
from make_lc_weights import replace_with_dict2, make_cost_func

"""
Cost functions:
1. 1's in everything, baseline
2. Roadnetwork conf 1
3. Roadnetwork conf 2
4. Roadnetwork conf 3
5. Roadnetwork conf 4
6. Upsamle by 2, otherwise conf 1
7. Land cover 1
8. Land cover 2, with all roads 0.25
9. Slope, needs to find something smart here
10. Slope with upsamling 2x
11. protected areas =1000
12. protected areas not so much
"""

def make_new_costs(number, shape_raster, out_dir, admin_in, code="ADM0_A3",test_case="UGA"):
    out_dir = Path(out_dir).expanduser()
    admin = gpd.read_file(admin_in)
    aoi = admin[admin[code] == test_case]
    
    ntl_big = rasterio.open("Data/ntl_annual/ntl_annual.tif")
    dest_crs = ntl_big.crs

    coords = [json.loads(aoi.to_json())["features"][0]["geometry"]]
    ntl, affine = mask(dataset=ntl_big, shapes=coords, crop=True, nodata=0)
    dest_shape = ntl.shape
    
    ups_targ = rasterio.open(f"Data/ups_targets/{test_case}.tif")
    ups_arr = ups_targ.read(1)
    ups_shape = ups_arr.shape
    
    
    if number==1:
        roads_raster, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi, f"Data/targets_filt/{test_case}.tif", 1, 1, 1, 1, 1, 1, 1,1)
        gf.save_raster(out_dir / f"{test_case}{number}.tif", roads_raster, affine)
    elif number==2:
        roads_raster, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi, f"Data/targets_filt/{test_case}.tif", 1/8, 1/6, 1/5, 1/3, 1/2, 1, 2/3,3/4)
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", roads_raster, affine, nodata=-1)
    elif number==3:
        roads_raster, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi, f"Data/targets_filt/{test_case}.tif", 1/10, 1/9, 1/8, 1/7, 1/6, 1/5, 1/4,1/3)
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", roads_raster, affine, nodata=-1)
    elif number==4:
        roads_raster, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi, f"Data/targets_filt/{test_case}.tif", 1/10, 1/10, 1/10, 1/5, 1/5, 1/2, 1/2,1/2)
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", roads_raster, affine, nodata=-1)
    elif number==5:
        roads_raster, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi, f"Data/targets_filt/{test_case}.tif", 1/4, 1/6, 1/8, 1/8, 1/6, 1/5, 1/5,1/2)
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", roads_raster, affine, nodata=-1)
    elif number==6:
        roads_raster, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi,  f"Data/ups_targets/{test_case}.tif", 1/10, 1/9, 1/8, 1/7, 1/6, 1/5, 1/4,1/3)
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", roads_raster, affine, nodata=-1)
    elif number==7:
        my_dict = make_cost_func("Data/LandCover_rs/UGA.tif", "Data/LandCover_rs/KEN.tif", "Data/classification/Uganda_grid.tif", "Data/classification/kenya_grid/kenya_grid.tif")
        land_cover = rasterio.open("Data/LandCover_rs/UGA.tif")
        land_cover = land_cover.read(1)
        roads_raster = replace_with_dict2(land_cover, my_dict)
        
        roads_raster2, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi, f"Data/targets_filt/{test_case}.tif", 1/10, 1/9, 1/8, 1/7, 1/6, 1/5, 1/4,1/3)
        dest_arr = np.multiply(roads_raster, roads_raster2)
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", dest_arr, affine, nodata=-1)
    elif number==8:
        my_dict = make_cost_func("Data/LandCover_rs/UGA.tif", "Data/LandCover_rs/KEN.tif", "Data/classification/Uganda_grid.tif", "Data/classification/kenya_grid/kenya_grid.tif")
        land_cover = rasterio.open("Data/LandCover_rs/UGA.tif")
        land_cover = land_cover.read(1)
        roads_raster = replace_with_dict2(land_cover, my_dict)
        
        roads_raster2, affine = prepare_roads(f"Data/costs_vec/africa/{test_case}.gpkg", aoi, f"Data/targets_filt/{test_case}.tif", 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4,1/4)
        dest_arr = np.multiply(roads_raster, roads_raster2)
        
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", dest_arr, affine, nodata=-1)
    elif number==9:
        arr, aff, crs = gf.clip_raster("Data/DEM/global.tif", aoi)
        dest_arr = make_same_as_dem(arr, aff, crs, dest_shape, affine, dest_crs)
        
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", dest_arr, affine, nodata=-1)
    elif number==10:
        arr, aff, crs = gf.clip_raster("Data/DEM/global.tif", aoi)
        dest_arr = make_same_as_dem(arr, aff, crs, ups_shape, affine, dest_crs)
        
        gf.save_raster(out_dir/ f"{test_case}{number}.tif", dest_arr, affine, nodata=-1)
        
        
for i in range(1,10):
    make_new_costs(i, "Data/targets_filt", "Data/test_tries", "admin_boundaries.gpkg", code="ADM0_A3",test_case="UGA")
        

/home/andre/.conda/envs/gridfinder/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:
/home/andre/.conda/envs/gridfinder/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:
/home/andre/.conda/envs/gridfinder/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:
/home/andre/.conda/envs/gridfinder/lib/python3.9/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


AttributeError: 'NoneType' object has no attribute 'data'